## 检索默认值

In [2]:
cars_config = {'volvo': 'c1c', 'benz': 'B2B'}

# 传统的 hash_table 做法, 先判断 hash_table 中是否存在
my_car = None
if 'volvo' in cars_config:
    my_car = cars_config['volvo']
else:
    my_car = 'CCC'

In [4]:
# Pythonic
my_car = cars_config.get('volvo', 'ccc123') # 如果未检索到，则返回定义的默认值。没有定义默认值则返回 None

上述方法返回不存在的键值时，不会修改原 dict

## Default Dict

In [6]:
from collections import defaultdict

In [7]:
my_dict = defaultdict(list)
my_dict['missing'].append('apple')
my_dict['missing'].append('banana')
my_dict['missing'].append(123)
my_dict

defaultdict(list, {'missing': ['apple', 'banana', 123]})

In [14]:
%%timeit
s = 'ThissssidtThhhis' * 1000000
d = defaultdict(int)
for key in s:
    d[key] += 1
d

1.39 s ± 34.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
%%timeit
s = 'ThissssidtThhhis' * 1000000
d = {}
for key in s:
    d.setdefault(key, 0)
    d[key] += 1
d

1.81 s ± 14.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [20]:
def constant_factory(value):
    return lambda:value
d = defaultdict(constant_factory('<missing>'))
d.update(name='John', action='ran')
d

defaultdict(<function __main__.constant_factory.<locals>.<lambda>()>,
            {'name': 'John', 'action': 'ran'})

In [21]:
'%(name)s %(action)s to %(unknown)s' % d

'John ran to <missing>'

## switch-case
使用字典索引函数

## 字典推导
效率提示不大，但是优化了可读性

In [22]:
from dataclasses import dataclass

### 补充 dataclass

用来更方便的创建数据形的类，自动生成 init, repr, eq, lt 等方法

In [24]:
@dataclass
class User:
    name: str
    email:str

In [40]:
list_of_users = [User('bob', 'bob@local'), User('Jack', 'jack@local')]
list_of_users = [User(f'bob{i}', f'bob{i}@local')
                 for i in range(10000000)]

In [41]:
%%timeit
email_for_user = {}
for user in list_of_users:
    if user.email:
        email_for_user[user.name] = user.email

email_for_user

3.8 s ± 16 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [42]:
%%timeit temp = 0
# Pythonic
email_for_user = {user.name: user.email
                  for user in list_of_users if user.email}
email_for_user

3.78 s ± 30.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## ChainMap

可以用来记录默认配置，和用户配置。默认配置无法修改，新配置可以。

In [1]:
from collections import ChainMap

In [3]:
m1 = {'soccer': 100, 'basketball': 200, 'vollyball': 300}
m2 = {'tennis':400, 'golf':500, 'soccer': 600}

cmap = ChainMap(m1, m2)
cmap

ChainMap({'soccer': 100, 'basketball': 200, 'vollyball': 300}, {'tennis': 400, 'golf': 500, 'soccer': 600})

In [4]:
cmap['golf']

500

In [5]:
cmap['soccer']

100

In [6]:
cmap['pingpong'] = 700
cmap

ChainMap({'soccer': 100, 'basketball': 200, 'vollyball': 300, 'pingpong': 700}, {'tennis': 400, 'golf': 500, 'soccer': 600})

In [7]:
del cmap['soccer']
cmap

ChainMap({'basketball': 200, 'vollyball': 300, 'pingpong': 700}, {'tennis': 400, 'golf': 500, 'soccer': 600})

In [8]:
del cmap['soccer']

KeyError: "Key not found in the first mapping: 'soccer'"

In [9]:
cmap

ChainMap({'basketball': 200, 'vollyball': 300, 'pingpong': 700}, {'tennis': 400, 'golf': 500, 'soccer': 600})

## 只读字典
需要频繁变动的配置

不怎么变动的静态配置

Python 提供了 MappingProxyType(mapping)

未来并发场景下可以发挥大作用(读写分离)

In [1]:
from types import MappingProxyType

In [3]:
read_write_dict = {'benz': 1010, 'ferrari': 2000}
read_only_dict = MappingProxyType(read_write_dict)
print(read_only_dict)

{'benz': 1010, 'ferrari': 2000}


In [4]:
read_only_dict['benz'] = 1

TypeError: 'mappingproxy' object does not support item assignment

In [7]:
read_write_dict

{'benz': 1010, 'ferrari': 2000}

In [8]:
read_write_dict['benz'] = 3333

In [9]:
read_only_dict

mappingproxy({'benz': 3333, 'ferrari': 2000})

In [10]:
read_write_dict.pop('benz')

3333

In [11]:
read_write_dict

{'ferrari': 2000}

In [12]:
read_only_dict

mappingproxy({'ferrari': 2000})

In [13]:
del read_write_dict

In [14]:
read_only_dict

mappingproxy({'ferrari': 2000})

## 排序字典

Python3 会根据输入的顺序自动排序

In [15]:
d = {'a': 400, 'c':200, 'b':300, 'd': 100}
sorted(d.items())   # items 将 dict 的每个元素当作 元组实现

[('a', 400), ('b', 300), ('c', 200), ('d', 100)]

In [17]:
sorted(d.items(), key = lambda x: x[1])

[('d', 100), ('c', 200), ('b', 300), ('a', 400)]

In [18]:
import operator
sorted(d.items(), key=operator.itemgetter(1))

[('d', 100), ('c', 200), ('b', 300), ('a', 400)]

In [19]:
sorted(d.items(), key=lambda x: x[1], reverse=True)

[('a', 400), ('b', 300), ('c', 200), ('d', 100)]

## 合并dict配置

update, 新字典覆盖旧字典的 value

多字典合并，使用 ‘**’ 运算符（3.5版本以上）

In [20]:
fruit_price_map_gz = {'apple': 10.5, "orange": 8.8}
fruit_price_map_sh = {'banana': 2.8, 'orange': 9.5}

In [ ]:
merged_price_map = {}
merged_price_map.update(fruit_price_map_gz)
merged_price_map.update(fruit_price_map_sh)

## 漂亮的打印 dict


In [22]:
books_map = {'name': 'Python 游戏编程', 'author': 'Bobby',
             'stars': 5, 'publish_at': '2022-06-10',
             'info': {'alias': 'Python game programming', 'users': 1000}}

In [23]:
print(books_map)

{'name': 'Python 游戏编程', 'author': 'Bobby', 'stars': 5, 'publish_at': '2022-06-10', 'info': {'alias': 'Python game programming', 'users': 1000}}


In [24]:
from pprint import pprint
pprint(books_map, indent=4, sort_dicts=False)

{   'name': 'Python 游戏编程',
    'author': 'Bobby',
    'stars': 5,
    'publish_at': '2022-06-10',
    'info': {'alias': 'Python game programming', 'users': 1000}}


In [33]:
import json # 缺点，部分结构无法序列化
tmp = json.dumps(books_map, indent=5, sort_keys=False, ensure_ascii=False)
tmp

'{\n     "name": "Python 游戏编程",\n     "author": "Bobby",\n     "stars": 5,\n     "publish_at": "2022-06-10",\n     "info": {\n          "alias": "Python game programming",\n          "users": 1000\n     }\n}'

In [34]:
print(tmp)

{
     "name": "Python 游戏编程",
     "author": "Bobby",
     "stars": 5,
     "publish_at": "2022-06-10",
     "info": {
          "alias": "Python game programming",
          "users": 1000
     }
}


In [35]:
import yaml
print(yaml.dump(books_map, allow_unicode=True))

author: Bobby
info:
  alias: Python game programming
  users: 1000
name: Python 游戏编程
publish_at: '2022-06-10'
stars: 5



In [36]:
26*26

676

## 奇怪的表达式

In [37]:
{True:'apple', 1: 'orange', 1.0:'banana'}

{True: 'banana'}

In [1]:
from addict import Dict

class ConfigDict(Dict):

    def __missing__(self, name):
        raise KeyError(name)

    def __getattr__(self, name):
        try:
            value = super(ConfigDict, self).__getattr__(name)
        except KeyError:
            ex = AttributeError(f"'{self.__class__.__name__}' object has no "
                                f"attribute '{name}'")
        except Exception as e:
            ex = e
        else:
            return value
        raise ex

In [2]:
tmp_dict = {'a': 123, "yuuk": 456}

In [3]:
tmp_cfg_dict = ConfigDict(tmp_dict)

In [4]:
tmp_dict

{'a': 123, 'yuuk': 456}

In [6]:
tmp_cfg_dict['a']

123

In [7]:
tmp_cfg_dict.a

123

In [8]:
tmp_dict['b']

KeyError: 'b'

In [9]:
tmp_cfg_dict['b']

KeyError: 'b'

In [10]:
tmp_cfg_dict.b

AttributeError: 'ConfigDict' object has no attribute 'b'

In [11]:
type(tmp_cfg_dict)

__main__.ConfigDict

In [12]:
type(tmp_dict)

dict

In [13]:
t = tmp_cfg_dict.to_dict()

In [14]:
type(t)

dict

In [23]:
tmp_dict['b'] = 1

In [24]:
tmp_cfg_dict['b'] = 1

In [25]:
tmp_dict

{'a': 123, 'yuuk': 456, 'b': 1}

In [26]:
tmp_cfg_dict

{'a': 123, 'yuuk': 456, 'b': 1}